# Code for downloading rainfall data from NOAA radar

##### Note: Install the rasterio module using the command line, "conda install rasterio"



In [ ]:
# Import modules
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
#from rasterio import features as ft
from rasterio.warp import calculate_default_transform, reproject, Resampling


In [ ]:
# Obtain the current working directory

glob.os.getcwd()


In [ ]:
# User inputs

SOI = input('Enter station name')
lat_SOI = float(input('Enter latitude'))
lon_SOI = float(input('Enter longitude'))
interval = input('Enter data interval')
start =input('Enter start date in mm-dd-yyyy')
end =input('Enter end date in mm-dd-yyyy')
#email = input('Enter recipient email address')

In [ ]:
# Load the .csv file containing radar locations in Kansas

radar = pd.read_csv('radars.csv')

radar.head(13)
#radar.tail()

In [ ]:
# Creating a Haversine function for computing the distance between two points on the earth's surface

def haversine (lat1,lon1,lat2,lon2):
    " A Function for calculating the distance between two points on the earth's surface using the Haversine formula"
    
    import math
    
    R = 6371000  # radius of the earth in meters
    a = math.sin(math.radians((lat1-lat2)/2))**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(math.radians((lon1-lon2)/2))**2
    if a < 0:
        print('row',row,'col',col,':', 'Error: sqrt(a) is undefined, a is negative')
    else:
        b = 2* math.atan2(math.sqrt(a), math.sqrt(1-a))
        row_col = (row,col)
        d = round(R*b/1000,3) # converting the distance from m to km
        return d #,row_col
    


In [ ]:
# Calculating the distances between the station of interest (SOI) and selecting the nearest distance

# Calculating the distance between the stations

distances = []
for i in range(len(radar)):
    distances.append(haversine(lat_SOI,lon_SOI,radar.Latitude[i],radar.Longitude[i]))



In [ ]:
# Select the nearest radar station to SOI and its geographic coordinates

for i in range(len(distances)):
    if distances[i] == min(distances):
        idx_nearest = i
        nearest_distance = distances[i]
        nearest_name = radar.Radar_station[i]
        nearest_coordinates = [radar.Latitude[i], radar.Longitude[i]]
        

print('idx:',idx_nearest)
print('nearest distance:', nearest_distance,'km')
print('nearest radar:', nearest_name)
print('nearest station coordinates:',nearest_coordinates)


In [ ]:
# RAINFALL CODE STARTS

In [ ]:
# List of data files in the folder

data_files = [file for file in glob.glob("*conus.tif")]
data_files


In [ ]:
# Load a single file to explore the data information

data = rasterio.open(data_files[2])

print(data)

print(data.shape)
print(data.width)
print(data.height)
print(data.indexes)
print(data.bounds)
print(data.crs)


In [ ]:
# Save columns and rows as well as values less than 0.

rows,cols = data.shape
idx_no_us = data.read(1) < 0
rows, cols
idx_no_us



In [ ]:
rainfall_3d = np.ones([rows,cols,len(data_files)]) * np.nan
rainfall_3d.shape

In [ ]:
# Create empty matrix

rainfall_3d = np.ones([rows,cols,len(data_files[1:2])]) * np.nan

# Iterate and append rainfall for each day to build a 3D array
for count,filename in enumerate(data_files[1:2]):
    data = rasterio.open(filename)
    print(count,data)
    rainfall = data.read(1)
    rainfall[idx_no_us] = np.nan
    rainfall_3d[:,:,count] = rainfall
#data

In [ ]:
# Inspect resulting 3D array

haversine(39.1836,-96.5717,39.1279,-96.6156)

In [ ]:
# Compute cumulative rainfall for each single pixel

cum_rainfall = np.nansum(rainfall_3d, axis=2)
cum_rainfall = cum_rainfall * 25.4 # inches to millimeters

cum_rainfall[859, 1674]


In [ ]:
# Inspect dimensions and data within the cumulative sum

print(cum_rainfall.shape)
cum_rainfall[0:10,0:5]


In [ ]:
# Set to `NaN` all the pixels that are outside the PRISM US boundaries and are equal to zero

cum_rainfall[idx_no_us | (cum_rainfall == 0)] = np.nan


In [ ]:
# Plot Map of cummulative rainfall

plt.figure(figsize=(8,8))
plt.imshow(cum_rainfall, aspect='auto')
plt.colorbar(orientation="horizontal", pad=0.1, aspect=50, label="Cumulative Rainfall (mm)")
#plt.scatter(859, 1674, marker='*', color='b', s=100)
plt.gca().axes.get_xaxis().set_visible(True)
plt.gca().axes.get_yaxis().set_visible(True)
plt.show()


In [ ]:
# RAINFALL CODE
sx = rasterio.open(data_files[2]) 
sx.shape[1]
sx.crs

In [ ]:
# Loop through data to extract Coordinates for each pixel

map_coords = []
for row in range(sx.shape[0]):    
    for col in range(sx.shape[1]):        
        map_coords.append(sx.xy(row,col, offset ='ul'))

map_coords[1:5]


In [ ]:
# A function to convert the coordinates reference system 

def reproject_et(inpath, outpath, new_crs):
    
    'A function to convert a raster from one coordinate system to another coordinate system. This function will load back into into python the new file created'
    'Definition of inputs: inpath = name of the file which is to be converted'
    'outpath = name of the new file after the conversion'  
    'new_crs = the new refrence system you want'
   
    dst_crs = new_crs # CRS for web meractor 

    with rasterio.open(inpath) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(outpath, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
                

In [ ]:
# Convert the data into the new coordinte system

new_crs = 'EPSG:4326'
inpath = data_files[2]
outpath = 'data_new.tif'
reproject_et(inpath,outpath,new_crs)

new_file = rasterio.open(outpath) # open the new file to python
new_file.crs
new_file.shape



In [ ]:
# Extract the geographic coordinates

map_coords = []
for row in range(new_file.shape[0]):    
    for col in range(new_file.shape[1]):        
        map_coords.append(new_file.xy(row,col, offset ='ul'))

map_coords[3:5]


In [ ]:
# Compute the distance between SOI and the varous coordinates

distances = []
for row in range(new_file.shape[0]):
    for col in range(new_file.shape[1]):
        distances.append(haversine(lat_SOI,lon_SOI,row,col))
        
        
       

In [ ]:
for i in range(len(distances)):
    if distances[i] == min(distances):
        idx_nearest = distances(i)
        nearest_distance = distances[i]
        #nearest_name = radar.Radar_station[i]
        nearest_coordinates = newfile.xy(row[i], col[i])
        

print('idx:',idx_nearest)
print('nearest distance:', nearest_distance,'km')
print('nearest radar:', nearest_name)
print('nearest station coordinates:',nearest_coordinates)


In [ ]:
# Find the dimension of each pixel

height_spacing = (new_file.bounds.top - new_file.bounds.bottom)/new_file.height
print('height spacing: ',height_spacing,'degrees')

width_spacing = (new_file.bounds.right- new_file.bounds.left)/new_file.width
print('width spacing: ',width_spacing,'degrees')


In [ ]:
# Find and create a mask for the missing data values 

nodata_value = new_file.nodata # find the missing data 

new_file_masked = np.ma.masked_equal(new_file.read(),nodata_value) # mask the missing data 

# data_masked[1,609,800] #[1,600,800]

In [ ]:
# Plot a map of the observed data without missing values

plt.figure(figsize= (6,8))
plt.imshow(new_file_masked[1,:,:], cmap = 'RdYlBu') #Band 1 contains the observed data

#plt.scatter(859, 1674, marker='*', color='b', s=100) 

cb = plt.colorbar(ticks = range(0,201,20), label = 'Total rainfall in contiguous USA for 2019 water year', orientation = 'horizontal')
plt.tight_layout()
#plt.savefig('ConUS_wyr')
plt.show()
